# 1.load package

In [ ]:
import json
from collections import namedtuple, Counter

import numpy as np
import scanpy as sc
import pandas as pd
import squidpy as sq

## Image manipulation and geometry
from tifffile import imread
from skimage.io import imread as skimread

## Plotting imports
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize, to_hex, Colormap
from matplotlib.cm import ScalarMappable
from matplotlib.colorbar import ColorbarBase
from matplotlib.lines import Line2D
from matplotlib import rc_context

In [ ]:
### Functions for registration of Xenium to Visium data and associated analyis and visualization
## Visualization functions
from companion_functions import (
    hexlist_to_cmap,
    polygons_coords_to_patch_collection,
    plot_polygons,
    hex_corner_offset,
    polygon_corners,
    celltypes,
    celltypes,
    hex_codes,
    ctype_to_code_map,
    ctype_hex_map,
    ctype_cmap,
)

## Analysis functions
from companion_functions import (
    unique_encode,
    get_xenium_to_morphology_transform_from_xenium_morphology_image,
    get_xenium_capture_polygon_um,
    transform_coordinates,
    get_median_spot_to_spot_distance_from_centroids,
    generate_space_filling_visium_polygons,
    get_visium_capture_polygon,
    __OUTSIDE_VISIUM_CAPTURE_AREA_BARCODE__,
    bin_xenium_data_to_visium_spots,
    generate_anndata_matrix_from_transcript_assignments,
)

# 2 load data and QC

In [ ]:
adata = sc.read_10x_h5("P:/PI/PI_Chuva_de_Sousa_Lopes/susana/Fu/Project/16_xenium/4st_Analysis of CD45 positive cell/outs/cell_feature_matrix.h5")
df = pd.read_csv("P:/PI/PI_Chuva_de_Sousa_Lopes/susana/Fu/Project/16_xenium/4st_Analysis of CD45 positive cell/outs/cells.csv.gz")

df.set_index(adata.obs_names, inplace=True)
adata.obs = df.copy()
adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

In [ ]:
adata = adata[adata.obs["y_centroid"] >12302].copy()

In [ ]:
# alway have MemoryError
sc.pp.calculate_qc_metrics(adata, percent_top=(10, 20, 50, 150), inplace=True)

In [ ]:
cprobes = (
    adata.obs["control_probe_counts"].sum() / adata.obs["total_counts"].sum() * 100
)
cwords = (
    adata.obs["control_codeword_counts"].sum() / adata.obs["total_counts"].sum() * 100
)
print(f"Negative DNA probe count % : {cprobes}")
print(f"Negative decoding count % : {cwords}")

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(15, 4))

axs[0].set_title("Total transcripts per cell")
sns.histplot(
    adata.obs["total_counts"][adata.obs["total_counts"]<250],
    kde=False,
    ax=axs[0],
)

axs[1].set_title("Unique transcripts per cell")
sns.histplot(
    adata.obs["n_genes_by_counts"],
    kde=False,
    ax=axs[1],
)


axs[2].set_title("Area of segmented cells")
sns.histplot(
    adata.obs["cell_area"],
    kde=False,
    ax=axs[2],
)

axs[3].set_title("Nucleus ratio")
sns.histplot(
    adata.obs["nucleus_area"] / adata.obs["cell_area"],
    kde=False,
    ax=axs[3],

In [ ]:
sc.pp.filter_cells(adata, min_counts=5)
sc.pp.filter_genes(adata, min_cells=3)

# 3. dowstream analysis of whole sample 

## 3.1 tsne

In [ ]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.tsne(adata)
sc.tl.leiden(adata,resolution = 1.5)

In [ ]:
custom_palette = {
  "0" : "#49E2F6",   
  "2" : "#347928",
  "3" : "#FEEC37",
  "4" : "#ff002a",
  "5" :"#550686",
  "6" : "#E71BE6",
  "7" : "#00879D",
  "8" : "#C1A029",
  "9" : "#A2F852",
  "10" : "#002060",
  "11" : "#565DFD",
  "12" : "#10686f",
  "13" : "#F67280",   
  "14" : "#355C7D",
  "15" : "#99B898",
  "16" : "#FFBB00",
  "17" : "#FF6F61",
  "18" : "#6B5B95",
  "19" : "#88B04B",
  "20" : "#92A8D1",
  "21" : "#955251",
  "22" : "#009B77",
  "23" : "#DD4124"
}


In [ ]:
import pandas as pd


tsne_df = pd.DataFrame(tsne_coords, columns=['tSNE_1', 'tSNE_2'])
tsne_df['leiden'] = leiden_clusters.values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.scatterplot(data=tsne_df, x='tSNE_1', y='tSNE_2', hue='leiden', palette=custom_palette, s=5, edgecolor=None)
plt.title("t-SNE Plot with Leiden Clusters")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.legend(title="Leiden Cluster", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig("tSNE_adata_1.2_Leiden_Clusters.pdf", format="pdf")
plt.show()

# 3.2 DEG

In [ ]:
adata.layers["scaled"] = sc.pp.scale(adata, copy=True).X

In [ ]:
sc.tl.rank_genes_groups(adata, 
                        groupby="leiden", 
                        method="wilcoxon", 
                        pts=True,
                        )

In [ ]:
rank_genes_df = sc.get.rank_genes_groups_df(adata, group=None)
filtered_df = rank_genes_df[(rank_genes_df["pct_nz_group"] > 0.6) & (rank_genes_df["pvals_adj"] < 0.05)]
filtered_df.to_excel('figure_revised/9st/rank_genes_df_XCL_whole_cluster_filter.xlsx', index=False)

In [ ]:
top_genes = []
groups = adata.obs['leiden'].cat.categories

for group in groups:
    top_genes.append(
        sc.get.rank_genes_groups_df(adata, group).head(5)
    )

# 将结果合并成一个DataFrame
top_genes_df = pd.concat(top_genes)
gene_names = top_genes_df['names'].unique()

In [ ]:
z_score_data = adata[:, gene_names].to_df(layer="scaled")
#z_score_data = adata[:, gene_names].to_df()
z_score_data['leiden'] = adata.obs['leiden'].values


z_score_data.reset_index(inplace=True)
z_score_data = z_score_data.drop(columns=['index'])

grouped_mean = z_score_data.groupby("leiden").mean()
grouped_mean.to_excel("grouped_mean_df_adata_top5_zscore.xlsx", index=True)

# 4. downsteam analysis of PTPRC positive immune cell

In [ ]:
import pandas as pd
import scipy

expression_matrix = adata.X
if isinstance(expression_matrix, scipy.sparse.spmatrix):
    expression_matrix = expression_matrix.todense()
    
adata.obs['PTPRC_high_1'] = (df_expression_matrix['PTPRC'] >1).astype(int)

adata.obs['filter_immune_cd45_1'] = "2"

adata.obs.loc[
    (adata.obs['leiden'].isin(["5", "12"])) & (adata2.obs['PTPRC_high_1']== 1),
    'filter_immune_cd45_1'
] = "1"

immune_2 = adata[adata.obs['filter_immune_cd45_1'] == "1"].copy()

In [ ]:
sc.pp.normalize_total(immune_2, inplace=True)
sc.pp.log1p(immune_2)
sc.pp.neighbors(immune_2)
sc.tl.umap(immune_2)
sc.tl.tsne(immune_2)
sc.tl.leiden(immune_2,resolution = 1)

In [ ]:
#set color
custom_palette = {
    "0": "#E71BE6",
    "1": "#00FFFF",
    "2": "#009203",
    "3": "#FEEC37",
    "4": "#ff002a",
    "5": "#4fa9ff",
    "6": "#C1A029",
    "7": "#0000FF",
    "8": "#FF1493",
    "9": "#E6BEFF",
    "10": "#565DFD",
    "11": "#FF6347",
    "12": "#A2F852",
    
}

In [ ]:
# resolution=1
with rc_context({'figure.figsize': (5, 5)}):
     sc.pl.tsne(immune_2,color=[ "immune_1",], size=50,palette=custom_palette,save="immune_fillter_res_1.pdf")

In [ ]:
import scanpy as sc


genes = ["CD3D", "KLRD1", "CD79A", "CD163", "CD1C", "KIT","PTPRC","MZB1","IRF8"]


for gene in genes:
    with rc_context({'figure.figsize': (5, 5)}): 
        sc.pl.tsne(immune_2, color=gene, size=50, save=f"_{gene}.pdf")

In [ ]:
sc.tl.rank_genes_groups(immune2, 
                        groupby="leiden", 
                        method="wilcoxon", 
                        pts=True,
                        )

In [ ]:
rank_genes_df = sc.get.rank_genes_groups_df(immune2, group=None)
filtered_df = rank_genes_df[(rank_genes_df["pct_nz_group"] > 0.6) & (rank_genes_df["pvals_adj"] < 0.05)]
filtered_df.to_excel('figure_revised/9st/rank_genes_df_XCL_whole_cluster_filter.xlsx', index=False)

In [ ]:
top_genes = []
groups = immune2.obs['leiden'].cat.categories

for group in groups:
    top_genes.append(
        sc.get.rank_genes_groups_df(immune2, group).head(5)
    )

top_genes_df = pd.concat(top_genes)
gene_names = top_genes_df['names'].unique()

In [ ]:
z_score_data = adata[:, gene_names].to_df(layer="scaled")
#z_score_data = adata[:, gene_names].to_df()
z_score_data['leiden'] = adata.obs['leiden'].values


z_score_data.reset_index(inplace=True)
z_score_data = z_score_data.drop(columns=['index'])

grouped_mean = z_score_data.groupby("leiden").mean()
grouped_mean.to_excel("grouped_mean_df_adata_top5_zscore.xlsx", index=True)

# 5 cell charter

In [ ]:
import squidpy as sq
import cellcharter as cc
import scanpy as sc
from lightning.pytorch import seed_everything
seed_everything(0)

In [ ]:
adata.obs['condition'] = 'sample'
from cellcharter.tl import TRVAE
save_path = "./trained_trvae_model"
model = TRVAE.load(save_path, adata=adata,map_location='cpu')
adata.obsm['X_trVAE'] = model.get_latent(adata.X, adata.obs['condition'])

In [ ]:
adata.obsm['X_trVAE_12_1']=adata.obsm['X_trVAE']

In [ ]:
#CellCharter’s spatial clustering
sq.gr.spatial_neighbors(adata, coord_type='generic', delaunay=True)
cc.gr.remove_long_links(adata)

In [ ]:
cc.gr.aggregate_neighbors(adata, n_layers=7, use_rep='X_trVAE')

In [ ]:
gmm = cc.tl.Cluster(
    n_clusters=10, 
    random_state=12345,
    # If running on GPU
    #trainer_params=dict(accelerator='gpu', devices=1)
)
gmm.fit(adata, use_rep='X_cellcharter')
adata.obs['spatial_cluster'] = gmm.predict(adata, use_rep='X_cellcharter')
obs_data = adata.obs
selected_data = obs_data[['cell_id', 'spatial_cluster']].rename(columns={'spatial_cluster': 'group'})
selected_data.to_csv("./cell charter/repeat nomolrize expression model/spatial_lay10_cluster_10.csv", index=False, sep=",")

In [ ]:
enrichment_results = cc.gr.enrichment(
    adata=adata_cellcharter,
    group_key='spatial_cluster_7_10',
    label_key='leiden_1.5_updata',
    copy=True
)

enrichment_df = pd.DataFrame(enrichment_results['enrichment'])
enrichment_df.to_csv('enrichment_values_cellchater_7_10.csv', index=True)

In [ ]:
#for lay7_cluster 10
import pandas as pd

adata.obs['leiden_modified'] = adata.obs['leiden_modified'].astype(int)


adata.obs['spatial_immune#'] = adata.obs['spatial_cluster'].astype(str)


conditions = (adata.obs['spatial_cluster'].isin([2, 3, 6, 9])) & (adata.obs['leiden_modified'] > 30)
adata.obs.loc[conditions, 'spatial_immune'] = (
    adata.obs.loc[conditions, 'spatial_cluster'].astype(str) + "_" +
    adata.obs.loc[conditions, 'leiden_modified'].astype(str)
)


check_results = adata.obs.loc[conditions, ['spatial_cluster', 'leiden_modified', 'spatial_immune']]


In [ ]:
enrichment_results = cc.gr.enrichment(
    adata=adata_cellcharter,
    group_key='spatial_cluster_7_10',
    label_key='leiden_modified',
    copy=True
)

enrichment_df = pd.DataFrame(enrichment_results['enrichment'])
enrichment_df.to_csv('enrichment_values_cellchater_7_10_all_cluster_immune.csv', index=True)

# 6.neiborhood analysis

## 6.1 foe xenium data

In [ ]:
immune_dict = immune_2.obs['leiden'].to_dict()

adata.obs['immune_filter'] = adata.obs.index.map(lambda x: immune_dict.get(x, '30'))

In [ ]:

adata.obs["cl_immune"] = np.where(
    adata_.obs["immune_filter"] == 30,
    adata.obs["leiden"],
    adata.obs["immune_filter"]
)
adata.obs["cl_immune"] = adata.obs["cl_immune"].astype("category")

In [ ]:
adata.obs['cl_immune'] = adata.obs['cl_immune'].astype('category')
sq.gr.spatial_neighbors(adata, coord_type="generic", delaunay=True)
sq.gr.centrality_scores(adata, cluster_key="cl_immune")
sq.gr.co_occurrence(adata,cluster_key="cl_immune")
sq.gr.nhood_enrichment(adata, cluster_key="cl_immune")

In [ ]:
enrichment_data = adata.uns['cl_immune_nhood_enrichment']
enrichment_df = pd.DataFrame(enrichment_data['zscore'])
enrichment_df 

enrichment_data = adata.uns['cl_immune_nhood_enrichment']

if 'count' in enrichment_data:
    enrichment_df = pd.DataFrame(enrichment_data['zscore'])
else:
    raise KeyError("'count' not found in leiden_nhood_enrichment data.")

expected_clusters = adata.obs['cl_immune'].cat.categories  
enrichment_df.index = expected_clusters[:enrichment_df.shape[0]]
enrichment_df.columns = expected_clusters[:enrichment_df.shape[1]]

enrichment_df.to_excel("figure_revised/9st/enrichment_results_cl_immune_new.xlsx", index=True)


In [ ]:
import anndata as ad
import squidpy as sq
import pandas as pd
import os

output_dir = "figure_revised"
os.makedirs(output_dir, exist_ok=True)


follicle_types = spe_immune.obs["follicle_type"].unique()
follicle_types = [ft for ft in follicle_types if ft != "30"]


adata_dict = {}
nhood_enrichment_list = []

for follicle in follicle_types:
    print(f"Processing {follicle}...")


    spe_subset = spe_immune[spe_immune.obs["follicle_type"] == follicle].copy()


    sq.gr.spatial_neighbors(spe_subset, coord_type="generic", delaunay=True)


    sq.gr.centrality_scores(spe_subset, cluster_key="immune_filter")

 
    sq.gr.co_occurrence(spe_subset, cluster_key="immune_filter")


    sq.gr.nhood_enrichment(spe_subset, cluster_key="immune_filter")

    enrichment_data = spe_subset.uns['immune_filter_nhood_enrichment']

    if 'count' in enrichment_data:
        enrichment_df = pd.DataFrame(enrichment_data['count'])
    else:
        raise KeyError(f"'count' not found in immune1_nhood_enrichment data for {follicle}.")


    expected_clusters = spe_subset.obs['immune_filter'].cat.categories
    enrichment_df.index = expected_clusters[:enrichment_df.shape[0]]
    enrichment_df.columns = expected_clusters[:enrichment_df.shape[1]]

    
    enrichment_df.columns = [f"{follicle}_{col}" for col in enrichment_df.columns]

    
    nhood_enrichment_list.append(enrichment_df)

    
    adata_dict[follicle] = spe_subset

    print(f"Finished processing {follicle}, results saved.")

print("All follicle types processed successfully!")


combined_enrichment = pd.concat(nhood_enrichment_list, axis=1)


combined_enrichment.to_excel(os.path.join(output_dir, "combined_enrichment_matrix.xlsx"))


print(combined_enrichment)


## 6.2 for IMC data

In [ ]:
adata = sc.read_h5ad("P:/PI/PI_Chuva_de_Sousa_Lopes/susana/Fu/Project/16_xenium/4st_Analysis of CD45 positive cell/spe1/adata_spe.h5ad")

In [ ]:
meta=pd.read_excel("meta_updated.xlsx")
adata.obs['cl_immune'] = meta['cl_immune'].values
adata.obs['follicle_type_updata1'] = meta['follicle_type_updata1'].values

In [ ]:
adata.obs['cl_immune'] = adata.obs['cl_immune'].astype('category')
sq.gr.spatial_neighbors(adata, coord_type="generic", delaunay=True)
sq.gr.centrality_scores(adata, cluster_key="cl_immune")
sq.gr.co_occurrence(adata,cluster_key="cl_immune")
sq.gr.nhood_enrichment(adata, cluster_key="cl_immune")

In [ ]:
enrichment_data = adata.uns['cl_immune_nhood_enrichment']
enrichment_df = pd.DataFrame(enrichment_data['zscore'])
enrichment_df 

enrichment_data = adata.uns['cl_immune_nhood_enrichment']

if 'count' in enrichment_data:
    enrichment_df = pd.DataFrame(enrichment_data['zscore'])
else:
    raise KeyError("'count' not found in leiden_nhood_enrichment data.")

expected_clusters = adata.obs['cl_immune'].cat.categories  
enrichment_df.index = expected_clusters[:enrichment_df.shape[0]]
enrichment_df.columns = expected_clusters[:enrichment_df.shape[1]]

enrichment_df.to_excel("figure_revised/9st/enrichment_results_cl_immune_new.xlsx", index=True)

In [ ]:
adata.obs['immune1'] = adata.obs['immune1'].astype('category')
spe_immune = adata[adata.obs["immune1"] != 16].copy()
spe_immune.obs["follicle_type"].unique()

In [ ]:
import anndata as ad
import squidpy as sq
import pandas as pd
import os


output_dir = "spe"
os.makedirs(output_dir, exist_ok=True)


follicle_types = spe_immune.obs["follicle_type"].unique()
follicle_types = [ft for ft in follicle_types if ft != "1"]


adata_dict = {}
nhood_enrichment_list = []


for follicle in follicle_types:
    print(f"Processing {follicle}...")


    spe_subset = spe_immune[spe_immune.obs["follicle_type"] == follicle].copy()


    sq.gr.spatial_neighbors(spe_subset, coord_type="generic", delaunay=True)


    sq.gr.centrality_scores(spe_subset, cluster_key="immune1")


    sq.gr.co_occurrence(spe_subset, cluster_key="immune1")


    sq.gr.nhood_enrichment(spe_subset, cluster_key="immune1")


    enrichment_data = spe_subset.uns['immune1_nhood_enrichment']


    if 'count' in enrichment_data:
        enrichment_df = pd.DataFrame(enrichment_data['count'])
    else:
        raise KeyError(f"'count' not found in immune1_nhood_enrichment data for {follicle}.")


    expected_clusters = spe_subset.obs['immune1'].cat.categories
    enrichment_df.index = expected_clusters[:enrichment_df.shape[0]]
    enrichment_df.columns = expected_clusters[:enrichment_df.shape[1]]


    enrichment_df.columns = [f"{follicle}_{col}" for col in enrichment_df.columns]

    nhood_enrichment_list.append(enrichment_df)

 
    adata_dict[follicle] = spe_subset

    print(f"Finished processing {follicle}, results saved.")

print("All follicle types processed successfully!")

combined_enrichment = pd.concat(nhood_enrichment_list, axis=1)

combined_enrichment.to_excel(os.path.join(output_dir, "combined_enrichment_matrix.xlsx"))


# 7 mapping

In [ ]:
#all boundaries
xenium_cell_boundaries_df=pd.read_csv('P:/PI/PI_Chuva_de_Sousa_Lopes/susana/Fu/Project/16_xenium/4st_Analysis of CD45 positive cell/outs/cell_boundaries.csv.gz')

#boundaries of human ovary sample
merged_df = pd.merge(adata.obs, xenium_cell_boundaries_df, on='cell_id', how='inner')
xenium_cell_polygons = [
poly[['vertex_x', 'vertex_y']].values
    for _, poly in  merged_df.groupby('cell_id')
]

## 7.1 for follicle type

In [ ]:
## The hexcodes for the colors used .
## The celltypes are mapped to the colors for plotting convenience.
celltypes, hex_codes = np.array(
    sorted(
        [
        ["30", "#333333"],  
        ["ca", "#008000"],  
        ["ca_late", "#FF3030"], 
        ["healthy", "#800080"], 
        ["type1", "#FFA500"], 
        ["type2", "#00FFFF"],  
        ["type3", "#FFFF00"],  
        ],
        key=lambda x: x[0],
    )
).T
## used for converting list of celltypes to categorical.
ctype_to_code_map = {ctype: i for i, ctype in enumerate(celltypes)}
## used for converting list of celltypes to colors.
ctype_hex_map = dict(zip(celltypes, hex_codes))
## matplotlib colormap
ctype_cmap = hexlist_to_cmap("celltype_cmap", hex_codes)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.colors import Normalize
import numpy as np


fig = plt.figure(figsize=(12, 9))
ax = plt.gca()

fig.patch.set_facecolor('black')
ax.set_facecolor('black')




plot_polygons(
   xenium_cell_polygons,
    ax=ax,
    array=list(map(ctype_to_code_map.get, adata2.obs['follicletype'])),
    edgecolor="Black",  
    linewidth=0.01,     
    norm=Normalize(vmin=0, vmax=len(celltypes)),
    cmap=ctype_cmap,
)




ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


plt.gca().invert_yaxis()


ax.set_xticks([])
ax.set_yticks([])


plt.savefig('figure_revised/9st/wholesample_ROI_low_updata_color.png', dpi=600, bbox_inches='tight', pad_inches=0.1, facecolor='black')
plt.show()

## 7.2 for xenium cell cluster

In [ ]:
## The hexcodes for the colors used .
## The celltypes are mapped to the colors for plotting convenience.
celltypes, hex_codes = np.array(
    sorted(
        [
            ["0", "#FEEC37"],  
            ["1", "#347928"], 
            ["2", "#4B4949"], 
            ["3", "#009B77"], 
            ["4", "#FF002A"], 
            ["5", "#92A8D1"], 
            ["6", "#6B5B95"],  
            ["7", "#DD4124"],  
            ["8", "#C1A029"],  
            ["9", "#A2F852"],  
            ["10", "#565DFD"],
            ["11", "#E6BEFF"],
            ["12", "#E71BE6"], 
            ["13", "#0000FF"], 
            ["14", "#00FFFF"], 
            ["15", "#FF1493"], 
            ["16", "#FFBB00"], 
            ["17", "#FF6F61"], 
            ["18", "#6B5B95"], 
            ["19", "#88B04B"], 
            ["20", "#92A8D1"], 
            ["21", "#955251"],
            ["22", "#009B77"],
            ["23", "#DD4124"], 
            ["30", "#F5F5F5"],
        ],
        key=lambda x: x[0],
    )
).T
## used for converting list of celltypes to categorical.
ctype_to_code_map = {ctype: i for i, ctype in enumerate(celltypes)}
## used for converting list of celltypes to colors.
ctype_hex_map = dict(zip(celltypes, hex_codes))
## matplotlib colormap
ctype_cmap = hexlist_to_cmap("celltype_cmap", hex_codes)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.colors import Normalize
import numpy as np


fig = plt.figure(figsize=(12, 9))
ax = plt.gca()


fig.patch.set_facecolor('black')
ax.set_facecolor('black')



）
plot_polygons(
   xenium_cell_polygons,
    ax=ax,
    array=list(map(ctype_to_code_map.get, adata.obs['leiden_1.5_updata'])),
    edgecolor="Black",  
    linewidth=0.01,    
    norm=Normalize(vmin=0, vmax=len(celltypes)),
    cmap=ctype_cmap,
)




ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


plt.gca().invert_yaxis()


ax.set_xticks([])
ax.set_yticks([])


plt.savefig('figure_revised/9st/wholesample_CL_low.png', dpi=600, bbox_inches='tight', pad_inches=0.1, facecolor='black')
plt.show()

## 7.3 for xenium cell nich

In [ ]:
## The hexcodes for the colors used .
## The celltypes are mapped to the colors for plotting convenience.
celltypes, hex_codes = np.array(
    sorted(
        [
            ["0", "#FF6F61"], 
            ["1", "#355C7D"], 
            ["2", "#6B5B95"],  
            ["3", "#FEEC37"],
            ["4", "#92A8D1"],  
            ["5", "#A2F852"],  
            ["6", "#00FFFF"],  
            ["7", "#E6BEFF"],  
            ["8", "#FF002A"], 
            ["9", "#E71BE6"], 
        ],
        key=lambda x: x[0],
    )
).T
## used for converting list of celltypes to categorical.
ctype_to_code_map = {ctype: i for i, ctype in enumerate(celltypes)}
## used for converting list of celltypes to colors.
ctype_hex_map = dict(zip(celltypes, hex_codes))
## matplotlib colormap
ctype_cmap = hexlist_to_cmap("celltype_cmap", hex_codes)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.colors import Normalize
import numpy as np

fig = plt.figure(figsize=(12, 9))
ax = plt.gca()


fig.patch.set_facecolor('black')
ax.set_facecolor('black')



plot_polygons(
   xenium_cell_polygons,
    ax=ax,
    array=list(map(ctype_to_code_map.get, adata_cellcharter.obs['spatial_cluster_7_10'])),
    edgecolor="Black", 
    linewidth=0.01,     
    norm=Normalize(vmin=0, vmax=len(celltypes)),
    cmap=ctype_cmap,
)




ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


plt.gca().invert_yaxis()


ax.set_xticks([])
ax.set_yticks([])

plt.savefig('figure_revised/9st/wholesample_CN.png', dpi=900, bbox_inches='tight', pad_inches=0.1, facecolor='black')
plt.show()

## 7.4 for PTPRC positive cell

In [ ]:
## The hexcodes for the colors used .
## The celltypes are mapped to the colors for plotting convenience.
celltypes, hex_codes = np.array(
    sorted(
        [
            ["0", "#FF0000"],  # Red
            ["1", "#FFFF00"],  # Blue
            ["2", "#333333"],  # Green"#555555"
            ["3", "#FFFF00"],  # Yellow
            ["4", "#FF00FF"],  # Magenta
            ["5", "#00FFFF"],  # Cyan
            ["6", "#FFA500"],  # Orange
            ["7", "#800080"],  # Purple
            ["8", "#A52A2A"],  # Brown
            ["9", "#000000"],  # Black
            ["10", "#FFC0CB"], # Pink
            ["11", "#FFD700"], # Gold
            ["12", "#008080"], # Teal
            ["13", "#ADFF2F"], # GreenYellow
            ["14", "#FF4500"], # OrangeRed
            ["15", "#2E8B57"], # SeaGreen
            ["16", "#D2691E"], # Chocolate
            ["17", "#8B4513"], # SaddleBrown
            ["18", "#DC143C"], # Crimson
            ["19", "#4682B4"], # SteelBlue
            ["20", "#006400"], # DarkGreen
            ["21", "#8A2BE2"], # BlueViolet
            ["22", "#FF7F50"], # Coral
            ["30", "#F5F5F5"],
        ],
        key=lambda x: x[0],
    )
).T
## used for converting list of celltypes to categorical.
ctype_to_code_map = {ctype: i for i, ctype in enumerate(celltypes)}
## used for converting list of celltypes to colors.
ctype_hex_map = dict(zip(celltypes, hex_codes))
## matplotlib colormap
ctype_cmap = hexlist_to_cmap("celltype_cmap", hex_codes)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.colors import Normalize
import numpy as np


fig = plt.figure(figsize=(12, 9))
ax = plt.gca()


fig.patch.set_facecolor('black')
ax.set_facecolor('black')



plot_polygons(
   xenium_cell_polygons,
    ax=ax,
    array=list(map(ctype_to_code_map.get, adata.obs['filter_immune'])),
    edgecolor=None,  
    linewidth=0,    
    norm=Normalize(vmin=0, vmax=len(celltypes)),
    cmap=ctype_cmap,
)

for idx, polygon in enumerate(xenium_cell_polygons):
    leiden_value = adata.obs.iloc[idx]['filter_immune']
    if leiden_value in ['1']:
   
        centroid = np.mean(polygon, axis=0)  
        scaled_polygon = 2 * (polygon - centroid) + centroid 
        
  
        color = "#FFFF00" if leiden_value == '1' else 'green' 

        poly_patch = Polygon(scaled_polygon, closed=True, facecolor=color, edgecolor=None, linewidth=0)
        ax.add_patch(poly_patch)


ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


plt.gca().invert_yaxis()


ax.set_xticks([])
ax.set_yticks([])

plt.savefig('figure_revised/9st/wholesample_ptprc_filter_low.png', dpi=600, bbox_inches='tight', pad_inches=0.1, facecolor='black')
plt.show()

In [ ]:
## The hexcodes for the colors used .
## The celltypes are mapped to the colors for plotting convenience.
celltypes, hex_codes = np.array(
    sorted(
      [
    ["0", "#E71BE6"],  
    ["1", "#00FFFF"],   
    ["2", "#009203"],   
    ["3", "#FEEC37"],  
    ["4", "#ff002a"],   
    ["5", "#4fa9ff"],   
    ["6", "#C1A029"], 
    ["7", "#0000FF"],  
    ["8", "#FF1493"],   
    ["9", "#E6BEFF"],  
    ["10", "#565DFD"], 
    ["11", "#FF6347"], 
    ["12", "#A2F852"],  
   ],
        key=lambda x: x[0],
    )
).T
## used for converting list of celltypes to categorical.
ctype_to_code_map = {ctype: i for i, ctype in enumerate(celltypes)}
## used for converting list of celltypes to colors.
ctype_hex_map = dict(zip(celltypes, hex_codes))
## matplotlib colormap
ctype_cmap = hexlist_to_cmap("celltype_cmap", hex_codes)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.colors import Normalize
import numpy as np


fig = plt.figure(figsize=(12, 9))
ax = plt.gca()


fig.patch.set_facecolor('black')
ax.set_facecolor('black')




plot_polygons(
   xenium_cell_polygons,
    ax=ax,
    array=list(map(ctype_to_code_map.get,adata.obs["immune_filter"])),
    edgecolor="Black", 
    linewidth=0.01,    
    norm=Normalize(vmin=0, vmax=len(celltypes)),
    cmap=ctype_cmap,
)

for idx, polygon in enumerate(xenium_cell_polygons):
    leiden_value = adata.obs.iloc[idx]["immune_filter"]
    if leiden_value != '30':
    
        centroid = np.mean(polygon, axis=0) 
        scaled_polygon = 3 * (polygon - centroid) + centroid  
        
        
        color = ctype_cmap(ctype_to_code_map[leiden_value])

       
        poly_patch = Polygon(scaled_polygon, closed=True, facecolor=color, edgecolor=None, linewidth=0)
        ax.add_patch(poly_patch)


ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


plt.gca().invert_yaxis()


ax.set_xticks([])
ax.set_yticks([])


plt.savefig('figure_revised/9st/whole_sample_immune_x3_1500dpi.png', dpi=1500, bbox_inches='tight', pad_inches=0.1, facecolor='black')
plt.show()